### 📜 PDF Splitter by TOC

#### 🛠️ Описание
Скрипт автоматически делит PDF-файлы на разделы на основе оглавления (TOC) и сохраняет их в отдельные папки.

#### 🚀 Как работает
1. **Извлекает оглавление из PDF.**
2. **Разбивает документ на секции.**
3. **Сохраняет секции как отдельные PDF-файлы в соответствующих папках.**

✨ Удобный и простой инструмент для работы с PDF!

#### Необходимые библиотеки

In [1]:
import os
import re
import fitz

#### Функции для извлечения структуры

In [2]:
def extract_toc_from_pdf(pdf_path):
    """
    Извлекает оглавление (TOC) из PDF-файла.
    
    Args:
        pdf_path (str): Путь к PDF-файлу.
    
    Returns:
        list: Список оглавления, где каждая запись - это (уровень, заголовок, страница).
    """
    # Открываем PDF-файл
    doc = fitz.open(pdf_path)
    # Получаем оглавление
    toc = doc.get_toc()
    # Возвращаем список с уровнями, заголовками и номерами страниц
    return [(level, title.strip(), page) for level, title, page in toc]

def split_pdf_by_toc(pdf_path):
    """
    Делит PDF на секции на основе оглавления.
    
    Args:
        pdf_path (str): Путь к PDF-файлу.
    
    Returns:
        list: Список секций с информацией о заголовке, уровне и диапазоне страниц.
    """
    # Открываем PDF-файл
    doc = fitz.open(pdf_path)
    # Извлекаем оглавление
    toc = extract_toc_from_pdf(pdf_path)
    sections = []

    # Проходим по оглавлению и определяем диапазоны страниц
    for i in range(len(toc)):
        level, title, start_page = toc[i]
        # Определяем конечную страницу текущей секции
        end_page = toc[i + 1][2] - 1 if i + 1 < len(toc) else doc.page_count
        # Добавляем секцию в список
        sections.append({
            "level": level,
            "title": title,
            "start_page": start_page,
            "end_page": end_page
        })

    return sections

def clean_title(title):
    """
    Очищает и форматирует заголовок для использования в качестве имени файла.
    
    Args:
        title (str): Заголовок секции.
    
    Returns:
        str: Очищенный и обрезанный заголовок.
    """
    # Убираем недопустимые символы
    clean_title = re.sub(r'[<>:"/\\|?*]', '', title)
    # Заменяем пробелы на "_", обрезаем длину до 50 символов
    return re.sub(r'\s+', '_', clean_title).strip()[:50]

def save_pdf_section(doc, start_page, end_page, output_path):
    """
    Сохраняет определённую секцию PDF в новый файл.
    
    Args:
        doc (fitz.Document): Исходный PDF-документ.
        start_page (int): Начальная страница секции.
        end_page (int): Конечная страница секции.
        output_path (str): Путь для сохранения новой секции.
    """
    # Проверяем, корректен ли диапазон страниц
    if start_page > end_page:
        print(f"Пропуск раздела: некорректный диапазон ({start_page} > {end_page})")
        return

    # Создаём новый PDF-документ для секции
    section_doc = fitz.open()
    for page_num in range(start_page - 1, end_page):
        # Добавляем страницы из исходного документа
        section_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)

    # Сохраняем только если есть страницы
    if section_doc.page_count > 0:
        section_doc.save(output_path)
        print(f"Сохранён раздел: {output_path}")
    else:
        print(f"Пропущен пустой раздел: {output_path}")
    section_doc.close()

def create_structure_and_save(pdf_path, output_dir):
    """
    Создаёт структуру директорий и сохраняет разбитые секции PDF.
    
    Args:
        pdf_path (str): Путь к PDF-файлу.
        output_dir (str): Путь к выходной директории.
    """
    # Открываем PDF-файл
    doc = fitz.open(pdf_path)
    # Разделяем PDF на секции
    sections = split_pdf_by_toc(pdf_path)

    # Проходим по каждой секции
    for section in sections:
        # Очищаем заголовок для имени директории
        title = clean_title(section["title"])
        section_output_dir = os.path.join(output_dir, title)
        # Создаём директорию для секции
        os.makedirs(section_output_dir, exist_ok=True)

        # Сохраняем PDF-секцию в новую директорию
        output_pdf_path = os.path.join(section_output_dir, f"{title}.pdf")
        save_pdf_section(doc, section["start_page"], section["end_page"], output_pdf_path)

def process_directory(input_dir, output_dir):
    """
    Обрабатывает все PDF-файлы в папке, включая вложенные папки.
    
    Args:
        input_dir (str): Путь к входной директории.
        output_dir (str): Путь к выходной директории.
    """
    # Рекурсивно обходим все файлы во входной директории
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".pdf"):
                # Определяем относительный путь для вложенной структуры
                relative_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, relative_path)
                os.makedirs(output_subdir, exist_ok=True)

                # Обрабатываем текущий PDF-файл
                pdf_path = os.path.join(root, file)
                create_structure_and_save(pdf_path, output_subdir)

#### Основной запуск 

In [3]:
# Указываем входную и выходную директории
input_dir = "../../data/Base_Books/"  # Путь к папке с PDF-файлами
output_dir = "output"  # Путь к папке для сохранения результатов

# Проверяем, существует ли входная папка
if not os.path.exists(input_dir):
    raise ValueError(f"Input directory {input_dir} does not exist.")
# Создаём выходную папку, если её нет
os.makedirs(output_dir, exist_ok=True)

# Запускаем обработку
process_directory(input_dir, output_dir)

print('____' * 50)
print(f" \n Обработка завершена. Результаты сохранены в {output_dir}.")

Сохранён раздел: output/Обь/Книга 3/ОБОЗНАЧЕНИЯ_И_СОКРАЩЕНИЯ/ОБОЗНАЧЕНИЯ_И_СОКРАЩЕНИЯ.pdf
Сохранён раздел: output/Обь/Книга 3/ВВЕДЕНИЕ/ВВЕДЕНИЕ.pdf
Сохранён раздел: output/Обь/Книга 3/1_Общая_характеристика_целевого_состояния_бассейна/1_Общая_характеристика_целевого_состояния_бассейна.pdf
Сохранён раздел: output/Обь/Книга 3/2_Характеристики_целевого_состояния_отдельных_водн/2_Характеристики_целевого_состояния_отдельных_водн.pdf
Пропуск раздела: некорректный диапазон (7 > 6)
Сохранён раздел: output/Обь/Книга 3/3.1_Общие_положения/3.1_Общие_положения.pdf
Сохранён раздел: output/Обь/Книга 3/3.2_Расчетные_участки_бассейна_р._Обь/3.2_Расчетные_участки_бассейна_р._Обь.pdf
Сохранён раздел: output/Обь/Книга 3/3.3_Долгосрочные_целевые_показатели_качества_воды/3.3_Долгосрочные_целевые_показатели_качества_воды.pdf
Сохранён раздел: output/Обь/Книга 3/3.3.1_Долгосрочные_целевые_показатели_по_расчетным/3.3.1_Долгосрочные_целевые_показатели_по_расчетным.pdf
Сохранён раздел: output/Обь/Книга 3/3.3.2_П